# 570 Interim Deliverable 2 

## MIP formulation

#### Input data:  
$$\begin{aligned}
\text{Section information:}
&& I & = \text{{all course sections}} \\
&& student & = \text{{predicted number of students who will register for each section}} \\
&& PS & = \text{{number of sessions for each section}}  \\
&& TS & = \text{{the timelength of each session in hours (e.g., 1.5, 2, 3)}}  \\
&& I_2 & = \text{{sections with 2 sessions a week (i.e., MW and TH)}} \\
\\
\text{Classroom information:}
&& J & = \text{{classrooms}} \\
&& seat & = \text{{number of seats in each classroom}}  \\
\\
\text{Day-time information:}
&& P & = \text{{day of week (i.e., M T W H F)}}  \\
&& T & = \text{{start time of a section}} \\
\end{aligned}$$

#### Decision variable:
Let the binary decision variable $X_{ijpt}$ denotes whether we assign course section $i$ to classroom $j$ at day $p$ starting at time $t$.

#### Objective and constraints:
Our goal is to maximize the capacity utilization rate. The linear program is thus:
$$\begin{aligned}
\text{maximize} && \sum_{i\in I}\sum_{j\in J}\sum_{p\in P}\sum_{t\in T}{X_{ijpt}\frac{student_i}{seat_j}} \\
\text{subject to:} \\
\
\text{No start time conflict for classes:} && X_{ijpt} + X_{(I-i)jpt} & \le 1 && \forall i \in I, j \in J, p \in P, t \in T\\
\\
\text{No class conflict when a class is in session:} && X_{ijpt} + X_{i'jp(t+c)}& \le 1 && \forall i,i' \in I; j \in J; p \in P; t \in T; c \in \{1,2,...,TS_i*2-1\}\\
\\
\text{All required sessions must be scheduled:} && \sum_{j\in J}\sum_{p\in P}\sum_{t\in T}{X_{ijpt}} & = PS_i && \forall i \in I\\
\\
\text{Number of students must not exceed total seats:} && X_{ijpt} * student_i & \le seat_j && \forall i \in I, j \in J, p \in P, t \in T \\
\\
\text{No class scheduled  to last beyond school time (i.e., 9.30PM):} && X_{ijpt'} & = 0 && \forall i \in I, j \in J, p \in P, t' \in \{T_{max}-(TS_i*2)+2,...,T_{max}\} \\
\\
\text{Binding clause for courses with 2 sessions (MW):} && X_{ij'M't} & = X_{ij'W't} && \forall i \in I_2, j \in J, t \in T\\
\text{Binding clause for courses with 2 sessions (TH):} && X_{ij'T't} & = X_{ij'H't} && \forall i \in I_2, j \in J, t \in T\\
\\
\text{No Friday class for courses with 2 sessions:} && X_{ij'F't} & = 0 && \forall i \in I_2, j \in J, t \in T \\
\end{aligned}$$

# Python Code

In [1]:
import gurobipy as grb
import pandas as pd
import numpy as np
import datetime
from datetime import time
from datetime import timedelta
import math

# for the room capacity file i remove the first record because it is same as the 7th record and will make some error in next step
capacity=pd.read_excel("Marshall_Room_Capacity_Chart.xlsx")
course_enrollment=pd.read_excel("Marshall_Course_Enrollment_1516_1617.xlsx") # for student prediction
#course=pd.read_excel("Marshall_Course_Enrollment_1516_1617_small.xlsx")
#section_info=pd.read_excel("section information.xlsx")


## Extra data from original dataset

In [2]:
# choose 20153 term as sample
data=course_enrollment.loc[course_enrollment.loc[:,"Term"]==20153,("Course","Section","First Days","First Begin Time","First End Time","First Room","First Instructor")]
data.columns=["Course","Section","FirstDays","FirstBeginTime","FirstEndTime","FirstRoom","FirstInstructor"]
# we only use those M/T/W/H/F/MW/TH
data=data[(data.FirstDays=="M")|(data.FirstDays=="T")|(data.FirstDays=="W")|(data.FirstDays=="H")|(data.FirstDays=="F")|(data.FirstDays=="MW")|(data.FirstDays=="TH")]
data=data[(data.FirstRoom!="ONLINE")&(data.FirstRoom!="DEN@Viterbi")]
# remove the record that have nan in first begintime
data=data[(data.FirstBeginTime==data.FirstBeginTime)|(data.FirstEndTime==data.FirstEndTime)]
# remove the record that have nan in professor
data=data[data.FirstInstructor==data.FirstInstructor]
data=data.loc[:,("Course","Section","FirstDays","FirstBeginTime","FirstEndTime","FirstInstructor")]

In [3]:
# timelength
timelength=[]
for i in data.index:
    #print i
    #print data.loc[i,:]
    #print data.loc[i,:]
    time1=data.loc[i,"FirstEndTime"]
    #print type(time1)
    time2=data.loc[i,"FirstBeginTime"]
    minutediff=((time1.hour*60+time1.minute)-(time2.hour*60+time2.minute)+10)
    halfhour=minutediff/30
    if minutediff%30>0:
        halfhour+=1
    timelength.append(halfhour)

In [4]:
# pattern
pattern=[]
for i in data.index:
    day=data.loc[i,"FirstDays"]
    if (day=="M") or (day=="T") or (day=="W") or (day=="H") or (day=="F"):
        #print day
        pattern.append(1)
    elif (day=="MW") or (day=="TH"):
        #print day
        pattern.append(2)

Use the "section_info=section_info[section_info.timelength<=2]" below to control the size of dataset (change value of 2), i only tried small one

In [5]:
# build section_info table
section_info=pd.DataFrame({"course":data["Course"],"section":data["Section"],"pattern":pattern,"timelength":timelength,"FirstInstructor":data["FirstInstructor"]})
# we only use those timelength less than 1.5 hours
section_info=section_info[section_info.timelength<=3]
# build table of professor and sections
Prof=section_info["FirstInstructor"].unique()
professor={}
for i in Prof:
    professor[i]=section_info.loc[section_info.loc[:,"FirstInstructor"]==i,"section"]
print max(section_info["timelength"])
print len(section_info)
#print section_info.index
# for change the index, do not delete 
section_info.index=range(len(section_info))
# get how many sessions we have in this sechedule
numofsections=sum(section_info["pattern"])

3
135


In [6]:
# select a subset of section id 
section_info=section_info.loc[range(15),:]
Prof=section_info["FirstInstructor"].unique()
professor={}
for i in Prof:
    professor[i]=section_info.loc[section_info.loc[:,"FirstInstructor"]==i,"section"]
numofsections=sum(section_info["pattern"])
print numofsections


28


## Build variables

In [7]:
# What we need to extract from data
# prediction about student registeration for every section
# how many section we need to schedule
# the pattern, time length of each section
# classroom capacity

# Variables
# course name
C={}
index=0
for i in section_info["section"]:
    C[i]=section_info.loc[section_info.loc[:,"section"]==i,"course"].get(index)
    index+=1
# section of course
I=section_info["section"]
# classroom
J=capacity["Room"]
# pattern of session (less than catagories in origional dataset)
P=["Monday","Tuesday","Wednesday","Thursday","Friday"]
P=pd.Series(i for i in P)
# start time of session (from 8:00am to 9:00pm,so the real classtime is between 8:00am to 10:00pm)
T1={}
time = datetime.timedelta(hours=8,minutes=0, seconds=0)
for i in range(27):
    T1[str(time+timedelta(hours=0.5*i))]=i
T=pd.Series(i for i in range(27))

# students' registeration prediction
student={}
for i in I:
    sum1=sum(course_enrollment.loc[course_enrollment.loc[:,"Section"]==i,"Reg Count"])
    len1=len(course_enrollment.loc[course_enrollment.loc[:,"Section"]==i,"Reg Count"])
    student[i]=round(sum1/len1)
#student=pd.Series(i for i in student)

# seat
seat={}
for index,row in capacity.iterrows():
    seat[row["Room"]]=row["Size"]
#seat=pd.Series(i for i in seat)
    
# pattern of each section
PS={}
index=0
for i in I:
    PS[i]=section_info.loc[section_info.loc[:,"section"]==i,"pattern"].get(index)
    index+=1
#PS=pd.Series(i for i in PS)
    
# timelength of each section
TS={}
index=0
for i in I:
    TS[i]=section_info.loc[section_info.loc[:,"section"]==i,"timelength"].get(index)
    index+=1
#TS=pd.Series(i for i in TS)

In [8]:
# build the score table that indicate the distance between classroom
# same room 0
# same floor 1
# same building 2
# different building in same area 3
# different area 4
# area1 (ACC&BRI&HOH) 
# area2 (JFF&JKP)
classroom=capacity["Room"].tolist()
classroom.insert(0,"classroom")
distance=pd.DataFrame(columns=classroom)
distance["classroom"]=capacity["Room"].tolist()
distance=distance.set_index("classroom")
classroom=capacity["Room"].tolist()
area1=["ACC","BRI","HOH"]
area2=["JFF","JKP"]
for i in classroom:#row
    for j in classroom:#column
        if i==j: # same room 0
            #print "same room"
            distance.loc[i,j]=0
        elif i[0:4]==j[0:4]: # same floor 1
            distance.loc[i,j]=1
        elif i[0:3]==j[0:3]: # same building 2
            distance.loc[i,j]=2
        elif ((i[0:3] in area1)&(j[0:3] in area1)) | ((i[0:3] in area2)&(j[0:3] in area2)): # different building but same area (ACC&BRI&HOH) OR (JFF&JKP) 3
            distance.loc[i,j]=3
        else: # different area 4
            distance.loc[i,j]=4
pairroom=distance
for i in pairroom.index:
    for j in pairroom.columns:
        pairroom.loc[i,j]=0

## Model and Object

In [9]:
# Building model
mod=grb.Model()

# Decision Variable
X={}
for i in I:
    for j in J:
        for p in P:
             for t in T:
                    X[i,j,p,t]=mod.addVar(vtype=grb.GRB.BINARY, name='x[{0},{1},{2},{3}]'.format(i,j,p,t))


In [10]:
#build table with 3 dimensions professor, day of week, time slot
Y={}
for p in P:
     for t in T:
            for pr in Prof:
                section_room=[]
                #print professor[pr].values
                for i in professor[pr].values:
                    #print i
                    for j in J:
                        if X[i,j,p,t]>0:
                            section_room.append(i)
                            section_room.append(j)
                        else:
                            section_room=0
                Y[pr,p,t]=section_room 

In [11]:
for p in P:
    for t in T:
        for pr in Prof:
            a=Y[pr,p,t]
            if (a!=0)&(t+TS[a[0]]<27):
                b=Y[pr,p,t+TS[a[0]]]
                if (b!=0):
                    num=pairroom.loc[a[1],b[1]]
                    num+=1
                    pairroom.loc[a[1],b[1]]=num
                    #print a[1],b[1],pairroom.loc[a[1],b[1]]
                    

In [12]:
pairroom

,ACC201,ACC205,ACC236,ACC303,ACC306B,ACC310,ACC312,BRI202,BRI202A,BRI5,...,JFF416,JFF417,JKP102,JKP104,JKP110,JKP112,JKP202,JKP204,JKP210,JKP212
classroom,,,,,,,,,,,,,,,,,,,,,
ACC201,960,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACC205,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACC236,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACC303,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACC306B,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACC310,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACC312,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BRI202,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BRI202A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# Objective
# As i set the constraint1 below, so the section with 2 or 3 sessions will have higher weight here
# but if i do not set constraint1, we cannot promise other sections will not take the vacancy
# for example, if DSO570 assigned to M/W 12:00, namely X[DSO570,j,"MW",12:00]=1, but X[i,j,"M",12:00]=0 and X[i,j,"W",12:00]=0, 
# so other courses maight be assign to this time too
mod.setObjective((sum(X[i,j,p,t]*student[i]/seat[j] for i in I for j in J for p in P for t in T)/numofsections)-sum(pairroom.loc[m,n]*distance.loc[m,n] for m in classroom for n in classroom),sense=grb.GRB.MAXIMIZE)



## Constraints of Classroom Utilization

In [14]:
# constraints
I_twoday=pd.Series()
I_oneday=pd.Series()
for i in I:
    if PS.get(i)==1:
        I_oneday=I_oneday.append(pd.Series(i))
    else:
        I_twoday=I_twoday.append(pd.Series(i))

In [15]:
# constraint1
# in each time slot in each day every professor can only teach one session at the same time
constraint1={}
for t in T:
    for p in P:
        for prof in Prof:
            constraint1[t,p,prof]=mod.addConstr(sum(X[i,j,p,t] for i in professor[prof].values for j in J)<=1)

In [16]:
# new constraint2
# every timeslot in every classroom in a specific day, there will be only one or zero class
constraint2={}
for j in J:
    for t in T:
        for p in P:
            constraint2[j,t,p]=mod.addConstr(sum(X[i,j,p,t] for i in I)<=1)
            
# in the next (number of required sessions)*2 timeslot all of X should be 0
I2=I
for i1 in I:
    I2=I2.drop(pd.Index(I).get_loc(i1))
    for j in J:
        for t in T:
            for p in P:
                timeslots=int(TS[i1])
                if t<=(27-timeslots): # avoid a situation that three hours course is assigned to 8:00 pm
                    #print i1,j,p,t
                    constraint2[i1,j,t,p]=mod.addConstr((X[i1,j,p,t]+sum(X[i2,j,p,t+num] for i2 in I2 for num in range(0,timeslots)))<=1,name="")

In [17]:
print len(constraint2)

88440


In [18]:
# this constraint3 is only for those sections whose pattern is 2, which means they have two session every week.
constraint3={}
for i in I_twoday:
    for j in J:
        for t in T:
            constraint3[i,j,"M",t]=mod.addConstr(X[i,j,"Wednesday",t]== X[i,j,"Monday",t],name="") 
            # the section with two sessions must have same 0/1 in M and W
            
            constraint3[i,j,"T",t]=mod.addConstr(X[i,j,"Thursday",t] == X[i,j,"Tuesday",t],name="") 
            # the section with two sessions must have same 0/1 in M and W

In [19]:
# this constraint4 is to control that when we maximum the utilization rate,the number of student registered 
#for one section should less than the classroom capacity
constraint4={}
for i in I:
    for j in J:
        for p in P:
             for t in T:
                    constraint4[i,j,p,t]=mod.addConstr(X[i,j,p,t]*student[i]<=seat[j],name="")

In [20]:
# this constraint5 is to make sure that in all time in all classroom in all day, 
#for each section, sum(X)=required sessions
constraint5={}
for i in I:
    constraint5[i]=mod.addConstr(sum(X[i,j,p,t] for j in J for p in P for t in T)==PS.get(i),name="")    

In [21]:
#constraint6: No Friday class for MW and TH section
constraint6={}
for i in I_twoday:
    constraint6[i] = mod.addConstr(sum(X[i,j,"Friday",t] for j in J for t in T) == 0) 

In [22]:
#no class can last longer than the '9PM-9.30PM' block
constraint7={}
for i in I:
    constraint7[i] = mod.addConstr(sum(X[i,j,p,T.max()-ts] for j in J for p in P for ts in range(int(TS[i]))) == 0)

In [23]:
print "number of constrints "+str(len(constraint7)+len(constraint6)+len(constraint5)+len(constraint4)+len(constraint3)+len(constraint2)+len(constraint1))

number of constrints 209551


## Classroom Distance for back-to-back sessions

In [24]:
# build the score table that indicate the distance between classroom
# same room 0
# same floor 1
# same building 2
# different building in same area 3
# different area 4
# area1 (ACC&BRI&HOH) 
# area2 (JFF&JKP)
classroom=capacity["Room"].tolist()
classroom.insert(0,"classroom")
distance=pd.DataFrame(columns=classroom)
distance["classroom"]=capacity["Room"].tolist()
distance=distance.set_index("classroom")
classroom=capacity["Room"].tolist()
area1=["ACC","BRI","HOH"]
area2=["JFF","JKP"]
for i in classroom:#row
    for j in classroom:#column
        if i==j: # same room 0
            #print "same room"
            distance.loc[i,j]=0
        elif i[0:4]==j[0:4]: # same floor 1
            distance.loc[i,j]=1
        elif i[0:3]==j[0:3]: # same building 2
            distance.loc[i,j]=2
        elif ((i[0:3] in area1)&(j[0:3] in area1)) | ((i[0:3] in area2)&(j[0:3] in area2)): # different building but same area (ACC&BRI&HOH) OR (JFF&JKP) 3
            distance.loc[i,j]=3
        else: # different area 4
            distance.loc[i,j]=4

## Best solution output

In [25]:
mod.optimize()
mod.setParam('OutputFlag',False)

print('Optimal solution:',mod.ObjVal)

Optimize a model with 209551 rows, 89100 columns and 2258520 nonzeros
Variable types: 0 continuous, 89100 integer (89100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [9e-04, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -921599.5297
Presolve removed 205288 rows and 57396 columns
Presolve time: 3.66s
Presolved: 4263 rows, 31704 columns, 208248 nonzeros
Found heuristic solution: objective -921599.5155
Variable types: 0 continuous, 31704 integer (31704 binary)

Root relaxation: objective -9.215992e+05, 6391 iterations, 0.50 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    -921599.1754 -921599.18  0.00%     -    4s

Explored 0 nodes (6415 simplex iterations) in 4.73 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: -921599

In [ ]:
# print to check result
#for i in I:
 #   for j in J:
  #      for p in P:
   #         for t in T:
    #            if X[i,j,p,t].x>0:
     #               print("i ",i," j ",j," p ",p," t ",t," X[i,j,p,t] ",X[i,j,p,t].x," student[i] ",student[i]," seat[j] ",seat[j]," X[i,j,p,t]*student[i]/seat[j] ",X[i,j,p,t].x*student[i]/seat[j])

In [160]:
# get the small subset of J that have classes in this classroom
J1=[]
for j in J:
    if sum(X[i,j,p,t].x for i in I for p in P for t in T)>0:
        J1.append(j)
#print J1

In [161]:
# return a blank table
def blanktable():
    T1=[]
    blank=[]
    time = datetime.timedelta(hours=8,minutes=0, seconds=0)
    for i in range(27):
        T1.append(str(time+timedelta(hours=0.5*i)))
        blank.append("")
    schedule=pd.DataFrame({"# timeslot": range(27),
                           "timeslot":T1,
                           "Monday":blank,
                           "Tuesday":blank,
                           "Wednesday":blank,
                           "Thursday":blank,
                           "Friday":blank})
    sched_cols = ['# timeslot','timeslot','Monday','Tuesday','Wednesday','Thursday','Friday']
    schedule = schedule[sched_cols]
    schedule=schedule.set_index('# timeslot')
    return schedule

In [162]:
# put the optimal solution into our beautiful schedule table
timetable={}
for j in J1:
    #print "J is "+j
    #print "schedule"+j
    schedule1=blanktable()
    for i in I:
        for p in P:
            for t in T:
                if X[i,j,p,t].x>0:
                    for a in range(int(TS[i])):
                        schedule1.loc[t+a,p]=C[i]+" / "+str(i)
                        #print schedule1
    timetable["schedule"+j]=schedule1
    #print "timetable "+"schedule"+j
    #print timetable["schedule"+j]

In [163]:
output=pd.ExcelWriter("schedule_24_sections.xlsx")
for j in J:
    if sum(X[i,j,p,t].x for i in I for p in P for t in T)>0:
        timetable["schedule"+j].to_excel(output,sheet_name=j,index=False)
output.save()

In [29]:
mod.write("mg.lp")

In [164]:
numerof1=0
for j in J:
    for i in I:
        for p in P:
            for t in T:
                if X[i,j,p,t].x>0:
                    ##print i
                    numerof1+=1
print numerof1

44
